# Config 클래스를 활용한 모드 전환

In [82]:
class Config:
    enable_backprop = True

In [83]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [84]:
def no_grad():
    return using_config('enable_backprop', False)

In [85]:
import numpy as np

In [86]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

In [87]:
class Variable:
    __array_priority__ = 200 # 연산자 우선순위

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 # 세대를 기록한다(부모 세대 + 1).

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop() # 함수를 가져온다.

            # 수정 전: gys = [output.grad for output in f.outputs] # ❶
            gys = [output().grad for output in f.outputs] # ❶
            gxs = f.backward(*gys) # ❷
            if not isinstance(gxs, tuple): # ❸
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # ❹
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    # +=을 쓰지 않는 이유: 값을 덮어쓰는게(인플레이스) 아니라 복사하게끔 + 사용
                    # 인플레이스 연산:  연산에 대한 결괏값을 새로운 변수에 저장하는 것이 아닌
                    # 기존 데이터를 대체하는 것을 의미
                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None # y는 약한 참조(weakref)


    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return f'variable({p})'

    def __mul__(self, other):
        return mul(self, other)

In [88]:
import weakref

class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)        # ❶ 별표를 붙여 언팩
        if not isinstance(ys, tuple): # ❷ 튜플이 아닌 경우 추가 지원
            ys = (ys,)

        outputs = [Variable(as_array(y)) for y in ys]  # Variable 형태로 되돌린다.

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs]) # 세대 설정
            for output in outputs:
                output.set_creator(self) # 출력 변수에 창조자를 설정한다.

            self.inputs  = inputs       # 입력 변수를 기억(보관)한다.
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [89]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

def square(x):
    f = Square()
    return f(x)

In [90]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

def exp(x):
    f = Exp()
    return f(x)

In [91]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

def add(x0, x1):
    x1 = as_array(x1)
    f = Add()
    return f(x0, x1)

Variable.__add__  = add
Variable.__radd__ = add

In [92]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy

def sub(x0, x1):
    x1 = as_array(x1)
    f = Sub()
    return f(x0, x1)

def rsub(x0, x1):
    x1 = as_array(x1)
    f = Sub()
    return f(x1, x0)

Variable.__sub__  = sub
Variable.__rsub__ = rsub

In [93]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

def mul(x0, x1):
    x1 = as_array(x1)
    f = Mul()
    return f(x0, x1)

Variable.__mul__  = mul
Variable.__rmul__ = mul

In [94]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1

def div(x0, x1):
    x1 = as_array(x1)
    f = Div()
    return f(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    f = Div()
    return f(x1, x0)

Variable.__truediv__  = div
Variable.__rtruediv__ = rdiv

In [95]:
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c - 1) * gy
        return gx

def pow(x, c):
    f = Pow(c)
    return f(x)

Variable.__pow__ = pow

In [96]:
class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy

def neg(x):
    f = Neg()
    return f(x)

Variable.__neg__  = neg

In [97]:
x = Variable(np.array(2.0))
y = -x # 부호를 바꾼다.
print(y)

variable(-2.0)


In [98]:
x = Variable(np.array(2.0))
y1 = 2.0 - x
y2 = x - 1.0
print(y1)
print(y2)

variable(0.0)
variable(1.0)


In [99]:
x = Variable(np.array(2.0))
y = x ** 3
print(y)

variable(8.0)
